In [2]:
from __future__ import print_function
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv3D, MaxPooling3D
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers
from matplotlib import pyplot as plt
from IPython.display import clear_output
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.models import load_model
%matplotlib inline
# Load the TensorBoard notebook extension.
%load_ext tensorboard
#from keras_radam import RAdam
import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn as sk
import time
import sys
import collections
# Keras turner imports
import kerastuner
from kerastuner.tuners import RandomSearch, BayesianOptimization 
from kerastuner.engine.hyperparameters import HyperParameters
import os

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [12]:
run_directory = "Z:\\FNS_GA_v5_100_gen_CNN_hyperparam_work\\models\\representativity\\models"
data_directory = "Z:\\FNS_GA_v5_100_gen_CNN_hyperparam_work\\models\\representativity\\data"
train_data_string = ["X_rep_training_data.csv.npy",
                     "Y_rep_training_data.csv.npy"]
validation_data_string = ["X_rep_validation_data.csv.npy",
                          "Y_rep_validation_data.csv.npy"]
previous_run_directory = "bayes_rep_1_"
prediction_type = 'representativity'

In [13]:
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tf.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.1.0
Keras Version: 2.2.4-tf

Python 3.7.9 (default, Aug 31 2020, 17:10:11) [MSC v.1916 64 bit (AMD64)]
Pandas 1.2.1
Scikit-Learn 0.24.1
GPU is available


In [14]:
#data_ = np.loadtxt("FNS_GA_Data_Runs_11_12_Paper_2.csv", delimiter=",")
#X = data_[:,0:150]
os.chdir(data_directory)
X_train = np.load(train_data_string[0])
Y_train = np.load(train_data_string[1])


#data_test = np.loadtxt("FNS_GA_Run_13_Data_Test.csv", delimiter=",")
X_test = np.load(validation_data_string[0])
Y_test = np.load(validation_data_string[1])

#X_test = np.load("test_2x2_fns_run_test_dat.npy")

In [15]:
from kerastuner import HyperModel
class HyperModel_555_v2(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape
    def build(self, hp):
        conv_size = hp.Int("first_conv_size", min_value=16, max_value=64, step=1)
        kernel_size = hp.Int("kernel_size", min_value=4, max_value=10, step=1)
        dense_layer_size = hp.Int("dense_layer_size", min_value=32, max_value=128, step=32)
        max_pool_val = hp.Int("max_pool_val", min_value=2, max_value=10, step=1)

        model = Sequential()
        model.add(Conv3D(conv_size, kernel_size=(kernel_size, 2, 2),
                         input_shape=self.input_shape, padding='same', data_format="channels_last", strides = (1, 1, 1)))
        model.add(LeakyReLU(alpha=0.3))
        model.add(BatchNormalization())
        ##model.add(Conv3D(32, (3, 3, 5), activation=LeakyReLU(alpha=0.3), padding='same'))

        for i in range(hp.Int("n_layers", 7, 12)):
            model.add(Conv3D(hp.Int(f"conv_{i}_size",
                                    min_value=16,
                                    max_value=64,
                                    step=8),
                             (kernel_size, 2, 2),
                             padding='same'))
            model.add(LeakyReLU(alpha=0.3))
            model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(max_pool_val, 2, 2)))
        model.add(Flatten())
        model.add(BatchNormalization())
        model.add(Dense(dense_layer_size))
        model.add(LeakyReLU(alpha=0.3))
        model.add(BatchNormalization())
        model.add(Dense(dense_layer_size))
        model.add(LeakyReLU(alpha=0.3))
        model.add(Dropout(hp.Float('dropout_value', min_value = 0.0, max_value = 0.4, step=0.1)))
        
        model.add(Dense(1, activation='linear'))

        ### Building the optimizer object
        #sgd = optimizers.SGD(lr = 0.1) # 1.0 0.0001 0.01
        opt = optimizers.Adam(lr = hp.Float(
                    'learning_rate',
                    min_value=0.0001,
                    max_value=0.04,
                    default=0.0001,
                    step=1e-4)) 
           
        ### Adding the loss function and optimizer to the model
        #mcp_save = ModelCheckpoint('CNN_3d_2x2_test_2.hdf5', save_best_only=True, monitor='val_loss', mode='min')
        #opt = RAdam(total_steps=100, warmup_proportion=0.1, min_lr=1e-5)
        model.compile(loss='mse', optimizer=opt, metrics=['mae'])
        ### Running the model


        return model

In [16]:
input_shape = (60, 2, 2, 4)
hypermodel = HyperModel_555_v2(input_shape)

In [17]:
os.chdir(run_directory)
LOG_DIR = previous_run_directory
max_trials_val = 100
# int(max_trials_val/10)
tuner_bo = BayesianOptimization(
            hypermodel,
            objective='val_mae',
            max_trials=max_trials_val,
            seed=1,
            executions_per_trial=2,
            num_initial_points = int(max_trials_val/10),
            directory = LOG_DIR)
##tuner_bo.search(x=X_train, y=Y_train,
##             epochs=300,
##             batch_size=2000,
##             validation_data=(X_test, Y_test))

INFO:tensorflow:Reloading Oracle from existing project bayes_rep_1_\untitled_project\oracle.json
INFO:tensorflow:Reloading Tuner from bayes_rep_1_\untitled_project\tuner0.json


In [18]:
#tuner_bo.results_summary()

In [19]:
#best_model = tuner_bo.get_best_models(num_models=10)
#best_model[0].save('best_model_bayes_test_keff_gen_46_2')

In [20]:
def train_top_models(models_list,
                     model_string_template = "test",
                     csv_logger_bool = True,
                     model_checkpoint_boo = True,
                     save_original_model = True,
                     train_models = True,
                     train_model_options = {}):
    
    for model_count, current_model in enumerate(models_list):
        model_string = model_string_template + "_model_" + str(model_count)
        
        callback_list = []
        if csv_logger_bool:
            csv_logger = CSVLogger(model_string + '.log')
            callback_list.append(csv_logger)
            
        if model_checkpoint_boo:
            checkpoint_filepath = model_string + "_chkpnt_.{epoch:02d}-{val_mae:.6f}.hdf5"
            model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                            filepath=checkpoint_filepath,
                                            monitor='val_mae',
                                            mode='min',
                                            save_best_only=True,
                                            verbose = 1)
            callback_list.append(model_checkpoint_callback)
            
        if save_original_model:
            current_model.save(model_string + '_original')
            
        if train_models:
            current_model.fit(train_model_options['x_train'],
                 train_model_options['y_train'],
                 validation_data=train_model_options['validation_data'],
                 epochs=train_model_options['epochs'],
                 batch_size=train_model_options['batch_size'],
                 verbose=train_model_options['verbose'],
                 callbacks=callback_list)

In [22]:
best_models = tuner_bo.get_best_models(num_models=10)
train_model_opt = collections.OrderedDict()
train_model_opt['x_train'] = X_train
train_model_opt['y_train'] = Y_train
train_model_opt['validation_data'] = (X_test, Y_test)
train_model_opt['epochs'] = 1000
train_model_opt['batch_size'] = int(len(X_train)/2)
train_model_opt['verbose'] = 1


train_top_models(best_models, train_model_options = train_model_opt,model_string_template = prediction_type)

INFO:tensorflow:Assets written to: representativity_model_0_original\assets
Train on 2948 samples, validate on 328 samples
Epoch 1/1000
1474/2948 [==============>...............] - ETA: 1s - loss: 0.0021 - mae: 0.0328
Epoch 00001: val_mae improved from inf to 0.02855, saving model to representativity_model_0_chkpnt_.01-0.028550.hdf5
2948/2948 [==============================] - 2s 765us/sample - loss: 0.0021 - mae: 0.0328 - val_loss: 0.0022 - val_mae: 0.0285
Epoch 2/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 0.0020 - mae: 0.0317
Epoch 00002: val_mae improved from 0.02855 to 0.02854, saving model to representativity_model_0_chkpnt_.02-0.028543.hdf5
2948/2948 [==============================] - 0s 126us/sample - loss: 0.0021 - mae: 0.0325 - val_loss: 0.0023 - val_mae: 0.0285
Epoch 3/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 0.0021 - mae: 0.0321
Epoch 00003: val_mae did not improve from 0.02854
2948/2948 [==============================] - 0s 10

1474/2948 [==============>...............] - ETA: 0s - loss: 0.0018 - mae: 0.0291
Epoch 00060: val_mae improved from 0.02854 to 0.02843, saving model to representativity_model_0_chkpnt_.60-0.028433.hdf5
2948/2948 [==============================] - 0s 123us/sample - loss: 0.0018 - mae: 0.0297 - val_loss: 0.0024 - val_mae: 0.0284
Epoch 61/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 0.0018 - mae: 0.0304
Epoch 00061: val_mae did not improve from 0.02843
2948/2948 [==============================] - 0s 102us/sample - loss: 0.0019 - mae: 0.0303 - val_loss: 0.0025 - val_mae: 0.0293
Epoch 62/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 0.0018 - mae: 0.0296
Epoch 00062: val_mae did not improve from 0.02843
2948/2948 [==============================] - 0s 103us/sample - loss: 0.0018 - mae: 0.0296 - val_loss: 0.0026 - val_mae: 0.0296
Epoch 63/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 0.0017 - mae: 0.0290
Epoch 00063: val_mae did not

Epoch 90/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 0.0017 - mae: 0.0285
Epoch 00090: val_mae did not improve from 0.02843
2948/2948 [==============================] - 0s 102us/sample - loss: 0.0017 - mae: 0.0289 - val_loss: 0.0024 - val_mae: 0.0285
Epoch 91/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 0.0017 - mae: 0.0288
Epoch 00091: val_mae did not improve from 0.02843
2948/2948 [==============================] - 0s 103us/sample - loss: 0.0017 - mae: 0.0288 - val_loss: 0.0026 - val_mae: 0.0310
Epoch 92/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 0.0015 - mae: 0.0269
Epoch 00092: val_mae did not improve from 0.02843
2948/2948 [==============================] - 0s 103us/sample - loss: 0.0016 - mae: 0.0283 - val_loss: 0.0027 - val_mae: 0.0311
Epoch 93/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 0.0016 - mae: 0.0278
Epoch 00093: val_mae did not improve from 0.02843
2948/2948 [========================

Epoch 120/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 0.0016 - mae: 0.0274
Epoch 00120: val_mae did not improve from 0.02843
2948/2948 [==============================] - 0s 102us/sample - loss: 0.0016 - mae: 0.0278 - val_loss: 0.0027 - val_mae: 0.0310
Epoch 121/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 0.0016 - mae: 0.0273
Epoch 00121: val_mae did not improve from 0.02843
2948/2948 [==============================] - 0s 103us/sample - loss: 0.0016 - mae: 0.0275 - val_loss: 0.0025 - val_mae: 0.0307
Epoch 122/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 0.0016 - mae: 0.0284
Epoch 00122: val_mae did not improve from 0.02843
2948/2948 [==============================] - 0s 103us/sample - loss: 0.0017 - mae: 0.0291 - val_loss: 0.0027 - val_mae: 0.0336
Epoch 123/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 0.0017 - mae: 0.0287
Epoch 00123: val_mae did not improve from 0.02843
2948/2948 [====================

Epoch 150/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 0.0015 - mae: 0.0275
Epoch 00150: val_mae did not improve from 0.02843
2948/2948 [==============================] - 0s 102us/sample - loss: 0.0016 - mae: 0.0275 - val_loss: 0.0026 - val_mae: 0.0303
Epoch 151/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 0.0015 - mae: 0.0269
Epoch 00151: val_mae did not improve from 0.02843
2948/2948 [==============================] - 0s 103us/sample - loss: 0.0015 - mae: 0.0266 - val_loss: 0.0028 - val_mae: 0.0343
Epoch 152/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 0.0016 - mae: 0.0276
Epoch 00152: val_mae did not improve from 0.02843
2948/2948 [==============================] - 0s 103us/sample - loss: 0.0016 - mae: 0.0272 - val_loss: 0.0025 - val_mae: 0.0319
Epoch 153/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 0.0015 - mae: 0.0271
Epoch 00153: val_mae did not improve from 0.02843
2948/2948 [====================

...

Epoch 880/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 3.2387e-04 - mae: 0.0132
Epoch 00880: val_mae did not improve from 0.01443
2948/2948 [==============================] - 0s 80us/sample - loss: 3.2545e-04 - mae: 0.0132 - val_loss: 6.2316e-04 - val_mae: 0.0158
Epoch 881/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 3.3781e-04 - mae: 0.0137
Epoch 00881: val_mae did not improve from 0.01443
2948/2948 [==============================] - 0s 79us/sample - loss: 3.4736e-04 - mae: 0.0138 - val_loss: 8.1536e-04 - val_mae: 0.0169
Epoch 882/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 3.8552e-04 - mae: 0.0139
Epoch 00882: val_mae did not improve from 0.01443
2948/2948 [==============================] - 0s 79us/sample - loss: 3.2799e-04 - mae: 0.0131 - val_loss: 7.2317e-04 - val_mae: 0.0165
Epoch 883/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 2.7924e-04 - mae: 0.0124
Epoch 00883: val_mae did not improve from 0.

Epoch 909/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 3.2876e-04 - mae: 0.0130
Epoch 00909: val_mae did not improve from 0.01443
2948/2948 [==============================] - 0s 80us/sample - loss: 3.7995e-04 - mae: 0.0145 - val_loss: 6.5251e-04 - val_mae: 0.0159
Epoch 910/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 3.1345e-04 - mae: 0.0130
Epoch 00910: val_mae did not improve from 0.01443
2948/2948 [==============================] - 0s 80us/sample - loss: 3.5376e-04 - mae: 0.0143 - val_loss: 7.4675e-04 - val_mae: 0.0170
Epoch 911/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 2.9630e-04 - mae: 0.0132
Epoch 00911: val_mae did not improve from 0.01443
2948/2948 [==============================] - 0s 79us/sample - loss: 3.1811e-04 - mae: 0.0134 - val_loss: 6.8815e-04 - val_mae: 0.0173
Epoch 912/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 3.6551e-04 - mae: 0.0143
Epoch 00912: val_mae did not improve from 0.

Epoch 938/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 2.6926e-04 - mae: 0.0121
Epoch 00938: val_mae did not improve from 0.01443
2948/2948 [==============================] - 0s 80us/sample - loss: 3.5013e-04 - mae: 0.0134 - val_loss: 0.0010 - val_mae: 0.0185
Epoch 939/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 3.2775e-04 - mae: 0.0132
Epoch 00939: val_mae did not improve from 0.01443
2948/2948 [==============================] - 0s 80us/sample - loss: 3.6134e-04 - mae: 0.0141 - val_loss: 6.8387e-04 - val_mae: 0.0157
Epoch 940/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 3.6464e-04 - mae: 0.0139
Epoch 00940: val_mae did not improve from 0.01443
2948/2948 [==============================] - 0s 79us/sample - loss: 4.2708e-04 - mae: 0.0151 - val_loss: 9.4508e-04 - val_mae: 0.0194
Epoch 941/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 3.1760e-04 - mae: 0.0134
Epoch 00941: val_mae did not improve from 0.0144

Epoch 967/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 2.8803e-04 - mae: 0.0129
Epoch 00967: val_mae did not improve from 0.01443
2948/2948 [==============================] - 0s 79us/sample - loss: 3.6078e-04 - mae: 0.0139 - val_loss: 0.0011 - val_mae: 0.0196
Epoch 968/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 2.7846e-04 - mae: 0.0121
Epoch 00968: val_mae did not improve from 0.01443
2948/2948 [==============================] - 0s 80us/sample - loss: 3.3924e-04 - mae: 0.0129 - val_loss: 9.6782e-04 - val_mae: 0.0196
Epoch 969/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 2.6978e-04 - mae: 0.0117
Epoch 00969: val_mae did not improve from 0.01443
2948/2948 [==============================] - 0s 79us/sample - loss: 3.0905e-04 - mae: 0.0128 - val_loss: 7.8442e-04 - val_mae: 0.0163
Epoch 970/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 3.0315e-04 - mae: 0.0128
Epoch 00970: val_mae did not improve from 0.0144

Epoch 996/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 2.9235e-04 - mae: 0.0124
Epoch 00996: val_mae did not improve from 0.01443
2948/2948 [==============================] - 0s 79us/sample - loss: 3.0889e-04 - mae: 0.0126 - val_loss: 8.8903e-04 - val_mae: 0.0195
Epoch 997/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 3.1706e-04 - mae: 0.0122
Epoch 00997: val_mae did not improve from 0.01443
2948/2948 [==============================] - 0s 78us/sample - loss: 3.0661e-04 - mae: 0.0127 - val_loss: 6.7583e-04 - val_mae: 0.0182
Epoch 998/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 3.2869e-04 - mae: 0.0134
Epoch 00998: val_mae did not improve from 0.01443
2948/2948 [==============================] - 0s 80us/sample - loss: 3.2184e-04 - mae: 0.0132 - val_loss: 0.0010 - val_mae: 0.0187
Epoch 999/1000
1474/2948 [==============>...............] - ETA: 0s - loss: 3.8688e-04 - mae: 0.0142
Epoch 00999: val_mae did not improve from 0.0144

In [ ]:
#loaded_model = keras.models.load_model("tf_model_1_chkpnt_.1737-1925532.000000.hdf5")
#print(loaded_model.summary())
#output_ = loaded_model.predict(X_test)
#for real, pred in zip(Y_test, output_):
#    print(real[0], pred[0])

In [ ]:

#train_model_opt = collections.OrderedDict()
#train_model_opt['x_train'] = X_train
#train_model_opt['y_train'] = Y_train
#train_model_opt['validation_data'] = (X_test, Y_test)
#train_model_opt['epochs'] = 3000
#train_model_opt['batch_size'] = int(len(X_train)/4)
#train_model_opt['verbose'] = 1
#
#
#train_top_models([loaded_model], model_string_template = "model_6_tf", train_model_options = train_model_opt)

In [ ]:
loaded_model = keras.models.load_model("model_6_tf_model_0_chkpnt_.707-0.000382.hdf5")
print(loaded_model.summary())
output_ = loaded_model.predict(X_test)
for real, pred in zip(Y_test, output_):
    print(real[0], pred[0])

In [14]:
import glob
import os

#list_of_files = glob.glob('*.hdf5') # * means all if need specific format then *.csv
#real_list = []
#for file in list_of_files:
#    if "model_7" in file:
#        real_list.append(file)
#latest_file = max(real_list, key=os.path.getctime)
#print(latest_file)

In [23]:
def get_latest_files(file_flag, X_data, Y_data, file_count_value = 10, file_suffix = '.hdf5'):
    
    list_of_files = glob.glob('*.hdf5') # * means all if need specific format then *.csv
    real_list = []
    list_of_best_models = []
        
    for file_count in range(file_count_value):
        file_flag_ = file_flag + str(file_count)
        #print(file_flag_)
        for file in list_of_files:
            if file_flag_ in file:
                real_list.append(file)
        latest_file = max(real_list, key=os.path.getctime)    
        #print(latest_file)
        list_of_best_models.append(latest_file)
        
    pred_data = []
    for model in list_of_best_models:
        loaded_model = keras.models.load_model(model)
        pred_data.append(loaded_model.predict(X_data))
        
    print_str = ''
    for model in list_of_best_models:
        print_str += model +","
    print(print_str)
        
    for count, _ in enumerate(pred_data[0]):
        print_str = ''
        for count_model, __ in enumerate(pred_data):
            #print(pred_data[count_model][0][0])
            print_str += str(pred_data[count_model][count][0]) + "," 
        print(str(Y_data[count][0]) + "," + print_str)

In [29]:
os.chdir("Z:\\FNS_GA_v4_50_gen_CNN_Training_3_24\\model_training\\total_flux\\models")
get_latest_files('tf_model_', np.load('test_variable_data_X.npy'),
                 np.load('test_variable_data_Y.npy'))

tf_model_0_chkpnt_.2985-2125741.000000.hdf5,tf_model_1_chkpnt_.1737-1925532.000000.hdf5,tf_model_2_chkpnt_.1907-1992067.125000.hdf5,tf_model_3_chkpnt_.2588-2109624.500000.hdf5,tf_model_4_chkpnt_.744-1944506.500000.hdf5,tf_model_5_chkpnt_.2931-2194857.750000.hdf5,tf_model_6_chkpnt_.893-2001896.875000.hdf5,tf_model_7_chkpnt_.2813-2168178.000000.hdf5,tf_model_8_chkpnt_.2601-2074682.250000.hdf5,tf_model_9_chkpnt_.2993-2229332.250000.hdf5,
0.00256,10268748.0,8434123.0,10859671.0,7916671.5,18989698.0,32949184.0,8188554.0,33778216.0,1079079.0,23380040.0,
0.00303,9214286.0,6777235.5,16369556.0,16173639.0,31490216.0,11039157.0,12935659.0,49348644.0,13497759.0,43659052.0,
0.0028,9278173.0,5406890.0,13716115.0,13211610.0,26549918.0,27635930.0,14462631.0,48962020.0,10683617.0,43259620.0,
0.00177,4225047.0,1737019.8,9511040.0,9075006.0,22979692.0,22328430.0,8546664.0,45207416.0,7552339.0,40028016.0,
0.00227,11917351.0,7551996.0,15263525.0,14607599.0,22672262.0,12083137.0,9437558.0,37619400.0,428423

0.00434,8569290.0,9158836.0,16001796.0,21000990.0,34372800.0,16513173.0,15748018.0,53802390.0,5800022.0,28566372.0,
0.00342,13671575.0,8993327.0,14631451.0,13534087.0,21773990.0,24323216.0,6753849.0,33595108.0,-53750.64,22271824.0,
0.00315,13339365.0,7345757.5,13920827.0,13310466.0,21491090.0,18186836.0,8327715.0,36126604.0,7476791.5,36761316.0,
0.00296,13808440.0,7978747.5,16514447.0,13508634.0,22611274.0,15177233.0,11009896.0,34506550.0,11960640.0,39886844.0,
0.00354,16082652.0,10903283.0,17916678.0,18420536.0,27928750.0,35046744.0,16731389.0,42153896.0,-328657.1,23268528.0,
0.00481,7003322.0,10413697.0,14765984.0,17070542.0,36748264.0,27009610.0,15143055.0,52300350.0,8021233.0,38948510.0,
0.00353,8440950.0,10386645.0,15970340.0,23537496.0,37211416.0,17384328.0,15489592.0,52729604.0,2477055.0,26480752.0,
0.00349,12672754.0,9162592.0,15703401.0,12419315.0,23204444.0,19650756.0,11790075.0,41138420.0,2991181.8,29842734.0,
0.00417,7982390.0,9086937.0,15637695.0,17760176.0,34373790.0,1599

0.00322,14842579.0,12918148.0,15664686.0,13078376.0,22977438.0,26162448.0,5717175.0,34883816.0,4872195.0,31879064.0,
0.00338,14756868.0,10021299.0,14659994.0,15016644.0,25958032.0,20704364.0,5814458.0,35275376.0,7584389.0,39594624.0,
0.003,13931943.0,5380293.5,14691200.0,13231271.0,25832444.0,16019695.0,8304371.0,38876252.0,10759100.0,54150450.0,
0.00323,17572806.0,12080713.0,17671154.0,16619851.0,27480400.0,14260417.0,6300169.5,38384570.0,14427757.0,41837440.0,
0.0026,12245090.0,6206766.5,12520651.0,11537940.0,20512320.0,25832020.0,5394741.0,33359352.0,9759097.0,40021760.0,
0.00188,17000298.0,10630633.0,17464502.0,14512434.0,23316770.0,33895710.0,7953750.5,37810840.0,4488466.0,31034154.0,
0.00406,15994914.0,11797668.0,16619447.0,12423610.0,24303976.0,17693324.0,7694821.5,38441040.0,18324434.0,51947828.0,
0.00171,14458413.0,11328116.0,9645244.0,13356024.0,18466370.0,30006214.0,3150714.8,34133604.0,13483985.0,41161630.0,
0.00218,18199950.0,11735893.0,12916935.0,14467087.0,20752692.0,979

0.00261,17606242.0,11068249.0,16514622.0,13983264.0,23047038.0,18293300.0,9745386.0,33661876.0,14565415.0,39262030.0,
0.00249,14365918.0,13249266.0,14374065.0,16718530.0,19346310.0,24758180.0,10967858.0,31688742.0,16919048.0,43580190.0,
0.00201,14064365.0,11510119.0,14497650.0,10541138.0,20504876.0,24922720.0,8201267.5,36507030.0,-184615.45,29386716.0,
0.00324,13337707.0,9815385.0,14307231.0,13285387.0,18726918.0,13515711.0,8966033.0,32273546.0,14320836.0,46532784.0,
0.00165,3915108.0,1091193.8,7229494.5,2955167.2,8924742.0,3801082.8,-179156.05,15494504.0,6143007.0,36346560.0,
0.00213,12982351.0,6800252.5,12986344.0,15578145.0,23780688.0,6053413.0,6430710.5,33885364.0,12174432.0,42420856.0,
0.00261,9796998.0,5551261.0,13758129.0,6633174.5,23861934.0,9859164.0,6359420.0,27329850.0,11403982.0,40459160.0,
0.00258,16236423.0,11008088.0,17294380.0,14375302.0,21367828.0,12940593.0,9834582.0,34360976.0,16098998.0,46766360.0,
0.00272,19127250.0,8596900.0,15259072.0,11902452.0,19286834.0,127469

0.00278,16095172.0,11463553.0,15215533.0,14681245.0,21191326.0,21590786.0,4560738.5,30298222.0,5882675.0,28414644.0,
0.00301,16918446.0,9620771.0,15478620.0,9482374.0,24654824.0,9282034.0,7426938.5,37150708.0,15066474.0,40489156.0,
0.00244,13517564.0,7848745.0,16042828.0,14397521.0,22788162.0,18066490.0,5153479.5,30235896.0,2470097.5,29478126.0,
0.00286,12614087.0,8253430.0,12644387.0,10025485.0,18876282.0,10662021.0,3756898.5,29704814.0,17909040.0,44062600.0,
0.00259,15761992.0,12022199.0,15530423.0,14204676.0,23122738.0,18541916.0,6304840.5,32899200.0,4349049.5,29858842.0,
0.00234,19242198.0,12096498.0,16769975.0,14131651.0,28343840.0,25969466.0,7787183.0,38553330.0,-4025377.5,25458694.0,
0.00241,6266568.5,3118216.5,7136010.0,7019833.5,14136326.0,11289842.0,1218841.8,21639596.0,9546806.0,42044932.0,
0.00214,11861859.0,9129247.0,14798077.0,11795917.0,20721878.0,19713378.0,3182131.0,30597932.0,4060651.2,28712294.0,
0.00278,12672203.0,4955757.5,14127459.0,10308804.0,22699418.0,6087739.0

0.0028,15552585.0,10201776.0,18413366.0,15743963.0,26860360.0,22042348.0,6109800.0,37494740.0,5810616.5,29989618.0,
0.00223,11637061.0,6587403.0,14191749.0,8083005.0,17142662.0,7646655.0,3065305.2,26463030.0,11295910.0,42838536.0,
0.00319,11808878.0,6946930.5,13753317.0,9091382.0,21843854.0,15620956.0,6777724.0,30716420.0,9531398.0,37078830.0,
0.00244,13179531.0,6068323.5,13611316.0,8947326.0,17918580.0,4131096.0,3508663.5,27468716.0,8246134.5,44479060.0,
0.00526,7505193.0,11426816.0,17618586.0,24422830.0,42168590.0,24160792.0,13464105.0,57051068.0,17140416.0,35855976.0,
0.00345,16235912.0,11849689.0,18530280.0,14570896.0,27053624.0,14514835.0,18653586.0,40944336.0,-3407776.8,22114122.0,
0.00407,17762686.0,12878225.0,18559382.0,15547096.0,27517566.0,23696158.0,10782415.0,36475732.0,3405267.0,31890282.0,
0.00357,18891104.0,13859639.0,18888986.0,15800276.0,30031428.0,19806046.0,10590223.0,40659956.0,2529078.2,33079092.0,
0.00154,9944170.0,6850759.0,14259088.0,8573376.0,17475992.0,1569260

0.00405,7709277.0,5483703.0,17011580.0,19666144.0,36527140.0,1612227.1,19586372.0,51643508.0,11686486.0,43339304.0,
0.00188,8668971.0,1612007.8,11207949.0,4859247.5,18565400.0,3591009.8,3869212.5,33159968.0,9155492.0,35676980.0,
0.00333,13822284.0,5683767.0,16318712.0,12887297.0,25324096.0,9941338.0,12596718.0,44974108.0,4930521.5,34281260.0,
0.0043,7912571.0,1005712.2,12581525.0,15388788.0,30715222.0,431035.62,15396802.0,48743616.0,5411475.0,41715296.0,
0.00363,6079314.0,-281904.16,11347845.0,14214282.0,28416746.0,6807493.0,12524911.0,47980104.0,-677140.44,39282440.0,
0.00262,12973539.0,9717770.0,15039687.0,11807740.0,21457994.0,29180764.0,10162420.0,35434724.0,-334024.3,23785104.0,
0.00511,9204334.0,5221271.0,17975040.0,16318017.0,35832270.0,10280756.0,15183913.0,48346510.0,-2697300.5,39973164.0,
0.00523,10407682.0,4117080.8,16608422.0,20230008.0,32038578.0,7353739.0,16012363.0,52340596.0,11289354.0,41433460.0,
0.00444,11554666.0,4555005.0,16117590.0,19292498.0,37579336.0,14924772.0,

0.00194,9405577.0,2583086.8,12155378.0,7470591.0,17407584.0,1548156.2,-256806.28,25896352.0,7775327.5,34995136.0,
0.00349,14122849.0,6424429.5,18733040.0,9492090.0,27076990.0,-2579517.5,13258906.0,35187864.0,4810410.5,41624796.0,
0.0014,7286406.0,-4170718.5,12420673.0,6724582.0,19464472.0,-1250609.2,2012599.2,33597016.0,2788362.0,31649376.0,
0.00264,5071596.0,3049811.5,14677496.0,17218822.0,34297640.0,13264783.0,12354345.0,50572988.0,-4599576.0,22380732.0,
0.00361,7820869.5,5720247.5,15034910.0,18459282.0,36624840.0,11975727.0,13560077.0,53920050.0,-114749.35,31400608.0,
0.00258,11069866.0,4200826.0,12401938.0,10826170.0,20586732.0,22621724.0,9667798.0,39203820.0,-1779501.4,19810036.0,
0.00176,8762799.0,5067456.0,10741161.0,9373148.0,14967067.0,15773714.0,-241310.42,19475962.0,3125103.5,26085106.0,
0.00371,8011284.0,8166139.5,15217333.0,16173098.0,32153712.0,18871582.0,16721348.0,52452740.0,11370887.0,32079412.0,
0.00352,16561516.0,12126754.0,16392572.0,12778568.0,24091292.0,24348998.0

0.00219,12154132.0,6854144.5,14305177.0,8184906.0,18964866.0,16365110.0,3079546.2,33353304.0,3456213.5,25308068.0,
0.00266,11010686.0,5285732.5,12249772.0,6125289.0,21523760.0,28618580.0,1050157.6,35014470.0,1256353.9,24301778.0,
0.00339,8810434.0,3159311.2,13535873.0,15967285.0,28439452.0,22922914.0,14145445.0,50033780.0,-4257580.0,15709263.0,
0.00355,15591536.0,7058639.5,15409144.0,7559250.0,25500626.0,22309646.0,9204769.0,33545432.0,-3993158.2,16730487.0,
0.00504,6898022.0,7441679.5,15659890.0,17176970.0,37641010.0,24022534.0,18883882.0,53524268.0,-328572.44,19930300.0,
0.00524,7804001.5,4819018.0,16349121.0,17615174.0,38486116.0,5386863.0,14977530.0,53649640.0,2830346.5,32379722.0,
0.00414,7089473.5,4062579.8,15292997.0,18899802.0,36793530.0,18012446.0,11206631.0,54869484.0,-257787.02,21850748.0,
0.00345,13886784.0,8990066.0,15731035.0,9405416.0,22224428.0,9857995.0,6996372.5,31211392.0,11703555.0,46547012.0,
0.0035,7109003.5,4424674.0,11672027.0,14143543.0,32887028.0,20592964.0,14

In [ ]:
loaded_model = keras.models.load_model("model_6_model_0_chkpnt_.338-0.007420.hdf5")
output_ =loaded_model.predict(X_train)
for real, pred in zip(Y_train, output_):
    print(real[0], pred[0])

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_mae', factor=0.5,
                              patience=150, min_lr=0.00001, verbose = 1)
csv_logger = CSVLogger('keff_long_training.log')

checkpoint_filepath = ''
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_mae',
    mode='min',
    save_best_only=True,
    verbose = 1)


In [ ]:
loaded_model.fit(X_train,
                 Y_train,
                 validation_data=(X_test, Y_test),
                 epochs=3000,
                 batch_size=1000,
                 verbose=1,
                 callbacks=[reduce_lr, csv_logger, model_checkpoint_callback])

In [ ]:
LOG_DIR = "bayes_test_rep_gen_46_4_100_epoch_"
max_trials_val = 100
# int(max_trials_val/10)
tuner_bo = BayesianOptimization(
            hypermodel,
            objective='val_mae',
            max_trials=max_trials_val,
            seed=1,
            executions_per_trial=2,
            num_initial_points = int(max_trials_val/10),
            directory = LOG_DIR)
tuner_bo.search(x=X_train, y=Y_train,
             epochs=100,
             batch_size=2000,
             validation_data=(X_test, Y_test))

In [ ]:
tuner.results_summary()

In [ ]:
best_model = tuner.get_best_models(num_models=1)

In [ ]:
best_model[0].summary()

In [ ]:
best_model[0].save('best_model_2_1_3ed_fns_5_mat')

In [ ]:
loaded_model = keras.models.load_model("best_model_2_1_3ed_fns_5_mat")

In [ ]:
loaded_model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=1000, batch_size=2500, verbose=1)

In [ ]:
test_pred = loaded_model.predict(X_test)

In [ ]:
outputfile = open("out_2.csv", "w")
for _ in test_pred:
    str_ =str(_[0])+"\n"
    outputfile.write(str_)
outputfile.close()